"""
案例:
    演示通过 朴素贝叶斯算法 实现  商品评论情感分析, 即: 好评, 差评...

朴素贝叶斯介绍:
    概述:
        贝叶斯: 仅仅依赖 概率 就可以进行分类的 一种机器学习算法.
        朴素:   不考虑特征之间的关联性, 即: 特征间都是相互独立的.
            原始:  P(AB) = P(A) * P(B|A) = P(B) * P(A|B)
            加入朴素后: P(AB) = P(A) * P(B)
    细节:
        因为我们分词要用到 jieba分词器, 记得先装一下, 例如: pip install jieba
"""

In [1]:
# 导包
import numpy as np                  # 数学计算包
import pandas as pd                 # 数据处理包
import matplotlib.pyplot as plt     # 画图包
import jieba                        # 分词包
from sklearn.feature_extraction.text import CountVectorizer # 词频统计包, 把评论内容 转成 词频矩阵.
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB               # 朴素贝叶斯对象


/home/hm/micromamba/envs/ml/lib/python3.9/site-packages/jieba/_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# 1. 读取文件, 获取到原始数据.
df = pd.read_csv('./data/书籍评价.csv', encoding='gbk')
# df.info()

In [3]:
# 2. 数据预处理.
# 2.1 添加labels列, 充当: 标签列.  好评 -> 1, 差评 -> 0
df['labels'] = np.where(df['评价'] == '好评', 1, 0)
# df.info()
# print(df)

# 2.2 抽取 labels列, 作为: 标签.
y = df['labels']

# 2.3 演示 jieba 分词
# print(jieba.lcut('好好学习, 天天向上! 我爱你你爱我, 蜜雪冰城甜蜜蜜! 小明骑车, 一把把把把住了.'))

# 2.4 对用户的评论信息, 做切词.
# 数据格式: [[第1条评论切词1, 切词2, 切词3...], [第2条评论切词1, 切词2, 切词3...], ...]
comment_list = [','.join(jieba.lcut(line)) for line in df['内容']]
# 数据格式: ['第1条评论切词1, 切词2, 切词3...', '第2条评论切词1, 切词2, 切词3...', ...]
print(comment_list)

# 演示字符串的 join()函数用法.
# my_list = ['aa', 'bb', 'cc']
# print(','.join(my_list))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.931 seconds.
Prefix dict has been built successfully.


[' ,从,编程,小白,的,角度看,，,入门,极佳,。', '很,好,的,入门,书,，,简洁,全面,，,适合,小白,。', '讲解,全面,，,许多,小,细节,都,有,顾及,，,三个,小,项目,受益匪浅,。', '前半部,分讲,概念,深入浅出,，,要言不烦,，,很赞', '看,了,一遍,还是,不会,写,，,有个,概念,而已', '中规中矩,的,教科书,，,零,基础,的,看,了,依旧,看不懂', '内容,太,浅显,，,个人,认为,不,适合,有,其它,语言,编程,基础,的,人', '破书,一本', '适合,完完全全,的,小白读,，,有,其他,语言,经验,的,可以,去,看,别的,书', '基础知识,写,的,挺,好,的,！', '太,基础', '略,_,嗦,。,。,适合,完全,没有,编程,经验,的,小白', '真的,真的,不,建议,买']


In [4]:
# 2.5 加载 停用词列表, 即: 里边记录的词, 不需要参与模型训练, 预测, 要被删除的词, 例如: 的, 啊, 哈, 从, 都...
with open('./data/stopwords.txt', 'r', encoding='utf-8') as src_f:
    # 2.5.1 一次读取所有的行
    stopwords_list = src_f.readlines()
    # 2.5.2 删除最后的 '\n'
    stopwords_list = [line.strip() for line in stopwords_list]
    # 2.5.3 对 停用词列表去重.
    stopwords_list = list(set(stopwords_list))
    #print(stopwords_list)

In [5]:
# 2.6 创建向量化对象, 从 评论切词列表(comment_list) 中 删除 停用词, 并且统计词频(单词矩阵).
transfer = CountVectorizer(stop_words=stopwords_list)   # 参数: 停用词列表.
# 2.7 统计词频矩阵, 先训练, 后转换, 在转数组.
# transfer.fit(comment_list)
# x的格式: [[第1条评论的切词分布, 有就是1, 没有就是0], [第2条评论的切词分布, 有就是1, 没有就是0], ...]
# x = transfer.transform(comment_list).toarray()
x = transfer.fit_transform(comment_list).toarray()
print(x)


[[0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
  0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
  0]
 [0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1
  1]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
  0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0
  0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 2 

In [6]:
# 2.8 看一下 我们13条评论, 切词, 且删除 停用词后, 一共剩下多少个词了.
print(transfer.get_feature_names_out())
print(len(transfer.get_feature_names_out()))    # 37个词, 即: 13条评论, 切词, 且删除 停用词后, 一共剩下多少个词了.


['一本' '一遍' '三个' '中规中矩' '依旧' '入门' '内容' '分讲' '前半部' '受益匪浅' '基础' '基础知识' '完完全全'
 '小白' '小白读' '建议' '很赞' '教科书' '有个' '极佳' '概念' '浅显' '深入浅出' '看不懂' '真的' '破书'
 '简洁' '细节' '经验' '编程' '要言不烦' '角度看' '讲解' '语言' '适合' '项目' '顾及']
37


In [7]:
# 2.9 因为就 13条数据, 我们把前10条当训练集, 后三条当测试集.
x_train = x[:10]
y_train = y[:10]

x_test = x[10:]
y_test = y[10:]

In [8]:
# 4. 模型训练.
estimator = MultinomialNB()     # 创建 朴素贝叶斯模型对象.
estimator.fit(x_train, y_train)

MultinomialNB()

In [9]:
# 5. 模型预测.
y_pred = estimator.predict(x_test)
print(f'模型预测结果: {y_pred}')


模型预测结果: [0 0 0]


In [10]:

# 6. 模型评估.
print(f'准确率: {accuracy_score(y_test, y_pred)}')

准确率: 1.0
